In [1]:
# Standar libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import cv2
import time
import os
import dpivsoft.Postprocessing as post
from scipy.spatial import distance
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

%matplotlib qt

# Set the Directories

In [2]:
dirResults = "./EPM_3992_results_piv16x16/0-2231_5digits/"
dirImages = "./EPM_3992_frame/"
extension = 'npz'
dt=1/60  # s

if not os.path.exists(dirResults):
    print("Unknown directory" )
else:
    files = os.listdir(dirResults)
    files = sorted([i for i in files if i.endswith(extension)])
    #print(files)

# Load background image (adjust filename as needed)
frame_path = dirImages + "frame_0000.jpg"
img = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)

print(len(files), " files found")

2231  files found


# Ploting single PIV field results

## Load single dpivsoft field

In [3]:
#Load PIV results
FrameToprocess = 0  # Adjust index as needed
time_stamp = int(FrameToprocess * dt)
filename = dirResults + files[FrameToprocess]  # Adjust index as needed
Data = np.load(filename)
x = Data['x']
y = Data['y']
u = Data['u']
v = Data['v']
velocity_magnitude = np.sqrt(u**2 + v**2)
X, Y, vorticity = post.vorticity(x, y, u, v, 'circulation')
velocity_divergence = post.divergence(x,y,u,v)


# Plot quiver


In [4]:
fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.quiver(x, y, u, v, scale=1,zorder=1)
ax1.set_xlabel('x (m)',fontsize=18)
ax1.set_ylabel('y (m)',fontsize=18)
ax1.imshow(img, cmap='gray', origin='upper', extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.3,zorder=2)
ax1.set_title('Instantenous velocity Field at '+str(time_stamp)+ 's',fontsize=18)
# Save the figure as PNG
figname=dirResults+'Figure_velocityVector_at_'+str(time_stamp)+'s.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')

## Plot velocity magnitude

In [5]:
# PLot vvelocity magnitude
print(filename)
vmin = 0 # Set your desired minimum value
vmax = 0.02 #velocity_magnitude.max() # Or set a custom maximum value

fig, ax = plt.subplots(figsize=(10, 6))
c = ax.pcolormesh(x, y, velocity_magnitude, shading='auto', cmap='inferno', vmin=vmin, vmax=vmax,zorder=1)
ax.imshow(img, cmap='gray', origin='upper', extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.5,zorder=2)
ax.set_xlabel('x (m)', fontsize=18)
ax.set_ylabel('y (m)', fontsize=18)
ax.set_title('Instantenous velocity magnitude at '+str(time_stamp)+ 's',fontsize=18)
# Make the colorbar the same height as the plot
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(c, cax=cax, label='Velocity Magnitude (u² + v²) in m/s at ' )
plt.show()
# Save the figure as PNG
figname=dirResults+'Figure_velocityMagnitude_at_'+str(time_stamp)+'s.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')

./EPM_3992_results_piv16x16/0-2231_5digits/gpu_field_00000.npz


## Plot vorticity

In [6]:
# PLot vorticity
X, Y, w = post.vorticity(x, y, u, v, 'circulation')
fig, ax = plt.subplots(figsize=(10, 6))
cf = ax.contourf(X, Y, w, 50, cmap='jet',zorder=1)
ax.imshow(img, cmap='gray', origin='upper', extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.5,zorder=2)
ax.set_xlabel('x (m)',fontsize=18)
ax.set_ylabel('y (m)',fontsize=18)
ax.set_title('Instantenous vorticity at '+str(time_stamp)+ 's',fontsize=18)
# Make the colorbar the same height as the plot
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(c, cax=cax, label='Vorticity')
plt.show()
# Save the figure as PNG
figname=dirResults+'Figure_vorticity_at_'+str(time_stamp)+'s.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')


/var/folders/0k/7cbvxwfn3tz4twsbtpvx93s40000gn/T/ipykernel_50293/2807639863.py:12: UserWarning: Adding colorbar to a different Figure <Figure size 2000x1200 with 2 Axes> than <Figure size 2000x1200 with 2 Axes> which fig.colorbar is called on.
  fig.colorbar(c, cax=cax, label='Vorticity')


## Plot divergence

In [7]:
# Plot divergence
flow_divergence = post.divergence(x,y,u,v)
fig, ax = plt.subplots(figsize=(10, 6))
cf = ax.contourf(x, y, flow_divergence, 50, cmap='jet',zorder =1)
ax.imshow(img, cmap='gray', origin='upper', extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.5,zorder=2)
ax.set_xlabel('x (m)',fontsize=18)
ax.set_ylabel('y (m)',fontsize=18)
ax.set_title('Instantenous divergence at '+str(time_stamp)+ 's',fontsize=18)
# Make the colorbar the same height as the plot
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(c, cax=cax, label='divergence')
plt.show()
# Save the figure as PNG
figname=dirResults+'Figure_divergence_at_'+str(time_stamp)+'s.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')


/var/folders/0k/7cbvxwfn3tz4twsbtpvx93s40000gn/T/ipykernel_50293/2651190018.py:12: UserWarning: Adding colorbar to a different Figure <Figure size 2000x1200 with 2 Axes> than <Figure size 2000x1200 with 2 Axes> which fig.colorbar is called on.
  fig.colorbar(c, cax=cax, label='divergence')


## Plot streamlines

In [8]:
# # Plot streamlines
# post.stream_lines(x,y,u,v)

## Extract velocity profile along a segment

In [9]:
print(filename)

data=velocity_magnitude
xdata=x
ydata=y
datamin = data.min()
datamax = data.max() 

vmin = 0 # Set your desired minimum value
vmax = 0.025 #velocity_magnitude.max() # Or set a custom maximum value

fig, ax = plt.subplots(figsize=(10, 6))
c = ax.pcolormesh(x, y, velocity_magnitude, shading='auto', cmap='inferno', vmin=vmin, vmax=vmax,zorder=1)
ax.imshow(img, cmap='gray', origin='upper', extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.5,zorder=2)
# Plot the velocity magnitude colormesh
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Select two points to plot magnitude along the line')
# Make the colorbar the same height as the plot
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(c, cax=cax, label='Velocity Magnitude (u² + v²) in m/s')


#Let user select two points
pts = plt.ginput(2)
plt.close(fig)

# Extract coordinates
(x0, y0), (x1, y1) = pts

# Number of points along the line
num_points = 200
x_line = np.linspace(x0, x1, num_points)
y_line = np.linspace(y0, y1, num_points)

# Interpolate magnitude along the line

magnitude_along_line = griddata((xdata.flatten(), ydata.flatten()), data.flatten(), (x_line, y_line), method='linear')
distances = distance.cdist([[x0, y0]], np.column_stack((x_line, y_line))).flatten()

# Define Gaussian function
def gaussian(x, a, x0, sigma, c):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2)) + c

# Remove NaNs for fitting
mask = ~np.isnan(magnitude_along_line)
x_fit = distances[mask]
y_fit = magnitude_along_line[mask]

# Initial guess for parameters: amplitude, center, width, offset
initial_guess = [y_fit.max(), x_fit[np.argmax(y_fit)], (x_fit.max()-x_fit.min())/4, y_fit.min()]

# Fit Gaussian
popt, pcov = curve_fit(gaussian, x_fit, y_fit, p0=initial_guess)



# Plot magnitude along the line
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(distances, magnitude_along_line, marker='o')
ax.plot(x_fit, gaussian(x_fit, *popt), 'r-', label='Gaussian Fit')
ax.set_xlabel('Distance along the chord (m)')
ax.set_ylabel('Velocity magnitude in m/s')
ax.set_title('Velocity profile at '+str(time_stamp)+'s') 
ax.grid(True)
ax.legend()
plt.show()

# Save the figure as PNG
figname=dirResults+'Figure_profileVelocity_at_'+str(time_stamp)+'s.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')

./EPM_3992_results_piv16x16/0-2231_5digits/gpu_field_00000.npz


# Calculate the mean values

In [10]:
files = os.listdir(dirResults)
files = sorted([i for i in files if i.endswith('.npz')]);
#print(files)

for i in range(len(files)):
    Data = np.load(dirResults+files[i])
    x = Data['x']
    y = Data['y']
    u = Data['u']
    v = Data['v']
    if i==0:
        xvort,yvort,vorticity=post.vorticity(x, y, u, v, 'circulation')
        magnitude_mean = np.zeros(u.shape)
        vorticity_mean = np.zeros(vorticity.shape)
    magnitude=np.sqrt(u**2 + v**2)
    xvort,yvort,vorticity=post.vorticity(x, y, u, v, 'circulation')
    magnitude_mean+= magnitude
    vorticity_mean+= vorticity

magnitude_mean/=len(files)  
vorticity_mean/=len(files)


## plot mean vorticity

In [11]:

data=vorticity_mean
xdata=xvort
ydata=yvort
datamin = data.min() # Set your desired minimum value
datamax = data.max() #velocity_magnitude.max() # Or set a custom maximum value

fig, ax = plt.subplots(figsize=(10, 6))
c = ax.pcolormesh(xdata, ydata, data, shading='auto', cmap='inferno', vmin=datamin, vmax=datamax, zorder =1)
ax.imshow(img, cmap='gray', origin='upper', extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.5,zorder=2)
ax.set_xlabel('x (m)', fontsize=18)
ax.set_ylabel('y (m)', fontsize=18)
ax.set_title('Mean Vorticity over '+str(len(files)*dt)+' s')
# Make the colorbar the same height as the plot
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(c, cax=cax, label='Vorticity')
# plt.show()
# Save the figure as PNG
figname=dirResults+'Figure_meanVorticity.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')

## plot mean velocity magnitude

In [12]:
data=magnitude_mean
xdata=x
ydata=y
datamin = data.min()# Set your desired minimum value
datamax = data.max() #velocity_magnitude.max() # Or set a custom maximum value

fig, ax = plt.subplots(figsize=(10, 6))
c = ax.pcolormesh(xdata, ydata, data, shading='auto', cmap='inferno', vmin=datamin, vmax=datamax,zorder=1)
ax.imshow(img, cmap='gray', origin='upper', extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.5,zorder=2)
ax.set_xlabel('x (m)', fontsize=18)
ax.set_ylabel('y (m)', fontsize=18)
ax.set_title('Mean velocity over '+str(len(files)*dt)+'s')
# Make the colorbar the same height as the plot
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(c, cax=cax, label='velocity(m/s)')
plt.show()
# Save the figure as PNG
figname=dirResults+'Figure_meanVelocity.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')

## extract mean velocity magnitude along segment

In [13]:
data=magnitude_mean
xdata=x
ydata=y
datamin = data.min()
datamax = data.max() 

fig, ax = plt.subplots(figsize=(10, 6))
c = ax.pcolormesh(xdata, ydata, data, shading='auto', cmap='inferno', vmin=datamin, vmax=datamax,zorder=1)
ax.imshow(img, cmap='gray', origin='upper', extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.5,zorder=2)
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Select two points to plot magnitude along the line')
# Make the colorbar the same height as the plot
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(c, cax=cax, label='Velocity Magnitude (u² + v²) in m/s')


#Let user select two points
pts = plt.ginput(2)
plt.close(fig)

# Extract coordinates
(x0, y0), (x1, y1) = pts

# Number of points along the line
num_points = 200
x_line = np.linspace(x0, x1, num_points)
y_line = np.linspace(y0, y1, num_points)

# Interpolate magnitude along the line

magnitude_along_line = griddata((xdata.flatten(), ydata.flatten()), data.flatten(), (x_line, y_line), method='linear')
distances = distance.cdist([[x0, y0]], np.column_stack((x_line, y_line))).flatten()

# Define Gaussian function
def gaussian(x, a, x0, sigma, c):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2)) + c

# Remove NaNs for fitting
mask = ~np.isnan(magnitude_along_line)
x_fit = distances[mask]
y_fit = magnitude_along_line[mask]

# Initial guess for parameters: amplitude, center, width, offset
initial_guess = [y_fit.max(), x_fit[np.argmax(y_fit)], (x_fit.max()-x_fit.min())/4, y_fit.min()]

# Fit Gaussian
popt, pcov = curve_fit(gaussian, x_fit, y_fit, p0=initial_guess)



# Plot magnitude along the line
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(distances, magnitude_along_line, marker='o')
ax.plot(x_fit, gaussian(x_fit, *popt), 'r-', label='Gaussian Fit')
ax.set_xlabel('Distance along the chord (m)')
ax.set_ylabel('Velocity magnitude in m/s')
ax.set_title('Mean velocity profile over '+str(len(files)*dt)+'s') 
ax.grid(True)
ax.legend()
plt.show()

# Save the figure as PNG
figname=dirResults+'Figure_profileMeanVelocity.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')

# Frequency analysis

## Define rectangular ROI 

In [14]:
print(dirResults)

data=magnitude_mean
xdata=x
ydata=y
datamin = data.min()
datamax = data.max() 


# Plot the velocity magnitude colormesh
fig, ax = plt.subplots(figsize=(10, 6))
#c = ax.pcolormesh(xdata, ydata, data, shading='auto', cmap='inferno', zorder = 1)
c = ax.pcolormesh(xdata, ydata, data, shading='auto', cmap='inferno', vmin=datamin, vmax=datamax,zorder=1)
ax.imshow(img, cmap='gray', origin='upper', extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.5,zorder=2)
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Select two corners of a rectangle')
# Make the colorbar the same height as the plot
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(c, cax=cax, label='Velocity Magnitude')


# Let user select two corners (diagonally opposite)
pts = plt.ginput(2)
(x1, y1), (x2, y2) = pts

# Draw rectangle on the original figure
rect = Rectangle((min(x1, x2), min(y1, y2)), abs(x2-x1), abs(y2-y1),
                 linewidth=2, edgecolor='cyan', facecolor='none')
ax.add_patch(rect)
plt.draw()

print(f"Rectangle corners: ({x1}, {y1}) and ({x2}, {y2})")

# Mask for points inside the rectangle
mask = (xdata >= min(x1, x2)) & (xdata <= max(x1, x2)) & (ydata >= min(y1, y2)) & (ydata <= max(y1, y2))

# Save the figure as PNG
figname=dirResults+'Figure_ROI.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')

./EPM_3992_results_piv16x16/0-2231_5digits/
Rectangle corners: (0.08345407981143466, 0.07168526182031253) and (0.10120513962606531, 0.0677661386633523)


## extract ROI averaged velocities and plot

In [15]:

files = os.listdir(dirResults)
print(dirResults)
files = sorted([i for i in files if i.endswith(extension)]);
uTS=[]
vTS=[]
t=[]
for i in range(len(files)):
    Data = np.load(dirResults+files[i])
    u = Data['u'][mask] 
    v = Data['v'][mask]
    uTS.append(np.mean(u))
    vTS.append(np.mean(v)) 
    t.append(i*dt)  # Assuming each file corresponds to a time step; adjust as needed     

magnitudeTS=np.sqrt(np.array(uTS)**2 + np.array(vTS)**2)
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(t, uTS, label='u component')
ax.plot(t, vTS, label='v component')
ax.plot(t, magnitudeTS, label='velocity magnitude')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Mean Velocity (m/s)')
ax.set_title('time series of the Velocity averaged over the ROI')
ax.legend()
plt.show()


# Save the figure as PNG
figname=dirResults+'Figure_timeseriesVelocity.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')

./EPM_3992_results_piv16x16/0-2231_5digits/


## Calculate the FFT

In [16]:
# Calculate FFT of uTS
import numpy as np

# t=np.asarray(t)
# uTS = np.sin(2*np.pi*t*1)  + 0.5*np.random.randn(len(t)) # Example signal: 1 Hz sine wave + noise
data=uTS
data_arr = np.array(data)-np.mean(data)  # Detrend by removing mean
N = len(data_arr)
freq = np.fft.rfftfreq(N, dt)
data_fft = np.fft.rfft(data_arr)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(freq, np.abs(data_fft))
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('FFT Amplitude')
ax.set_title('FFT of Mean u in ROI')
# Save the figure as PNG
figname=dirResults+'Figure_FFTVelocity.png'
fig.savefig(figname, dpi=300, bbox_inches='tight')